In [1]:
library(ggplot2)
library(dplyr)
library(stringr)
library(lubridate)
library(LaplacesDemon)
library(counties)
library(glmnet)
library(forecast)
library(zoo)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:dplyr’:

    intersect, setdiff, union


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘LaplacesDemon’


The following objects are masked from ‘package:lubridate’:

    dst, interval


Loading required package: Matrix

Loaded glmnet 3.0-2


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘forecast’


The following object is masked from ‘package:LaplacesDemon’:

    is.constant



Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [2]:
variableExpander <- function(keysAndVariable, unitKey, timeKey){ ### Expands into unit/time format
  
  orderedData <- keysAndVariable %>% arrange(get(unitKey), get(timeKey))
  
  outcomeName <- names(keysAndVariable)[!(names(keysAndVariable) %in% c(unitKey, timeKey))]
  
  outcomeMatrixForm <- matrix(NA, nrow=length(unique(keysAndVariable[, unitKey])), 
                              ncol=length(unique(keysAndVariable[, timeKey])), byrow=T)
  
  rownames(outcomeMatrixForm) <- unique(keysAndVariable[, unitKey])
  
  colnames(outcomeMatrixForm) <- str_replace_all(unique(keysAndVariable[, timeKey])[order(unique(keysAndVariable[, timeKey]))], pattern="-", replace="")
  
  for (index in 1:length(keysAndVariable[, unitKey])){
    
    outcomeMatrixForm[keysAndVariable[, unitKey][index], str_replace_all(keysAndVariable[, timeKey][index], pattern="-", replace="")] <- keysAndVariable[, outcomeName][index]
    
  }
  
  return(outcomeMatrixForm)
  
}

In [3]:
countyCovidData <- read.csv("/home/josh/GitHub/covid-19-data/us-counties.csv",
                           stringsAsFactors=F)

countyCovidData$fips <- as.character(countyCovidData$fips)

mandateData <- read.csv("state_policy_updates_20201129_0720.csv", stringsAsFactors = F)

stateAndFips <- read.csv('state and county fips.csv', stringsAsFactors=FALSE)

stateAndFips$fips <- as.character(stateAndFips$fips)


countyCovidData <- countyCovidData %>% mutate(date = ymd(date))

countyCovidData <- countyCovidData %>% mutate(week = week(date))

countyCovidData <- countyCovidData %>% mutate(county=tolower(county))


In [4]:
#mandateData %>% filter(state_id=='CA') %>% arrange(ymd(date))
### You'll have to find the corresponding stop date to every start date
### To do that, self_join on state_id, fips, and policy_type
### look for consecutive start/stops

### Another option is to not look too far into the future, so you don't have to worry about 
### start/stops

In [7]:
unique(mandateData$policy_type) ## Aggregate mandate types into codes

### Healthcare gathering related

healthcareGatheringMandates <- c('Suspended Elective Medical Dental Procedures', 'Allow Audio Only Telehealth', 'Nursing Homes',
                                'Medical', 'Nursing Home Visitations', 'Alternative Care Facilities')

healthcareResumeMandates <- c('Resumed Elective Medical Procedures')

### Healthcare non-gathering related

healthcareNonGatheringMandates <- c("Reopened ACA Enrollment Using a Special Enrollment Period", 
                                   'Modify Medicaid Requirements With 1135 Waivers Date Of CMS Approval',
                                   'Allow Expand Medicaid Telehealth Coverage')

### General Gathering related

generalGatheringMandates <- c("Gatherings", 'Public Gatherings', "Outdoor and Recreation", 'Gatherings',
                             'Graduation Ceremony Guidelines')

## Gathering business related 

businessMandates <- c("Entertainment", "Manufacturing", "Non-Essential Businesses", "Food and Drink",
                     'Day Care', 'Childcare (K-12)', 'Gyms', 'Wholesale Trade', 'Houses of Worship',
                     'Residential Overnight Camps', 'Personal Care', 'Travel', 'museums', 'libraries',
                     'Colleges & Universities', 'Construction', 'Bars', 'higher education', 'travel',
                     'training jobs', 'construction', 'travelor from out of state', 'Education',
                     'Day camps/overnight camps', 'Agriculture')

### non-gathering business related

businessNonGatheringMandates <- c('Stop Enforcement Of Evictions Overall Or Due To Covid Related Issues',
                                 'Order Freezing Utility Shut Offs','Froze Mortgage Payments',
                                 'Renter Grace Period Or Use Of Security Deposit To Pay Rent')

## Mask

maskMandates <- c("Mandate Face Mask Use By All Individuals In Public Spaces",
                 "Mask Requirement", "Mandate Face Mask Use By All Individuals In Public Facing Businesses")

### Shelter in place/stay at home

shelterMandates <- c('Shelter in Place', 'Quarantine')

### Misc
misc <- c('Public Health Advisory System', 'Phase 1', 'Phase 2', 'State of Emergency', 'Phase 3', 'Phase 4', 'Phase 5',
         'Health Risk Status', 'New Phase', 'Graduation', 'Election', 'State of Emergency/Funds',
         'Updated Guidelines', 'Executive Order', 'Phase 2, 3', 'Phase 2 Step 2')

[1] "Food and Drink"                                                      
 [2] "Day Care"                                                            
 [3] "Childcare (K-12)"                                                    
 [4] "Entertainment"                                                       
 [5] "Manufacturing"                                                       
 [6] "Gyms"                                                                
 [7] "Shelter in Place"                                                    
 [8] "Public Health Advisory System"                                       
 [9] "Phase 1"                                                             
[10] "Non-Essential Businesses"                                            
[11] "Phase 2"                                                             
[12] "Outdoor and Recreation"                                              
[13] "State of Emergency"                                                  
[14] "Suspended Elective Medical Dental Procedures"                        
[15] "Allow Audio Only Telehealth"                                         
[16] "Wholesale Trade"                                                     
[17] "Modify Medicaid Requirements With 1135 Waivers Date Of CMS Approval" 
[18] "Phase 3"                                                             
[19] "Allow Expand Medicaid Telehealth Coverage"                           
[20] "Reopened ACA Enrollment Using a Special Enrollment Period"           
[21] "Mandate Face Mask Use By All Individuals In Public Facing Businesses"
[22] "Medical"                                                             
[23] "Resumed Elective Medical Procedures"                                 
[24] "Stop Enforcement Of Evictions Overall Or Due To Covid Related Issues"
[25] "Phase 4"                                                             
[26] "Mask Requirement"                                                    
[27] "Houses of Worship"                                                   
[28] "Residential Overnight Camps"                                         
[29] "Phase 5"                                                             
[30] "Health Risk Status"                                                  
[31] "Gatherings"                                                          
[32] "Mandate Face Mask Use By All Individuals In Public Spaces"           
[33] "New Phase"                                                           
[34] "Nursing Homes"                                                       
[35] "Personal Care"                                                       
[36] "Public Gatherings"                                                   
[37] "Stop Initiation Of Evictions Overall Or Due To Covid Related Issues" 
[38] "Travel"                                                              
[39] "museums"                                                             
[40] "libraries"                                                           
[41] "Order Freezing Utility Shut Offs"                                    
[42] "Graduation"                                                          
[43] "Colleges & Universities"                                             
[44] "Election"                                                            
[45] "Froze Mortgage Payments"                                             
[46] "State of Emergency/Funds"                                            
[47] "Graduation Ceremony guidelines"                                      
[48] "Construction"                                                        
[49] "Updated Guidelines"                                                  
[50] "Renter Grace Period Or Use Of Security Deposit To Pay Rent"          
[51] "Bars"                                                                
[52] "higher education"                                                    
[53] "Executive Order"                           

In [ ]:
countyDataWithNewCases <- countyCovidData %>% group_by(state, county) %>%
mutate(new_cases = c(NA, diff(cases))) %>% arrange(state, county, date) %>% 
mutate(smoothed_new_cases = rollmean(new_cases, 7, na.pad = TRUE)) %>% arrange(state, county, date) %>% 
filter(month(ymd(date)) >= 3)

In [ ]:
wantedState <- 'Minnesota'
wantedCounty <- 'hennepin'

chosenCounty <- countyDataWithNewCases %>% filter(county==wantedCounty & state == wantedState)

In [ ]:
plot(chosenCounty$smoothed_new_cases, type='l')